### This notebook is basically scratch paper.  We were just playing around with the data and seeing what is there.  Additionally, we tested different methods and functions to see what they would do on this data.  Everything that we used to clean and process the data was copied to 01_datacleaning.

In [174]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import PorterStemmer
import re

In [17]:
# https://songhuiming.github.io/pages/2017/04/02/jupyter-and-pandas-display/
pd.options.display.max_rows
pd.set_option('display.max_colwidth', -1)

/Users/jsf/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


In [18]:
train = pd.read_csv('./data/train.csv')
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1


In [3]:
train.isna().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [4]:
train.shape

(7613, 5)

In [6]:
train.dtypes

id           int64
keyword     object
location    object
text        object
target       int64
dtype: object

In [23]:
train[train['location'].notna()].head()

,id,keyword,location,text,target
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/lHYXEOHY6C,1
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT http://t.co/YAo1e0xngw,0
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set ablaze in Aba. http://t.co/2nndBGwyEi,1
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS ABLAZE http://t.co/qqsmshaJ3N,0


In [8]:
train['keyword'].value_counts()

fatalities               45
armageddon               42
deluge                   42
harm                     41
damage                   41
                         ..
forest%20fire            19
epicentre                12
threat                   11
inundation               10
radiation%20emergency     9
Name: keyword, Length: 221, dtype: int64

In [11]:
train[train['keyword'].isna()]['target'].value_counts()

1    42
0    19
Name: target, dtype: int64

In [22]:
train['text'].str.contains('forest').sum()

23

In [83]:
train[(train['keyword'].notna()) & (train['text'].str.contains('forest fire'))]

,id,keyword,location,text,target
4020,5711,forest%20fire,NaN,e1.1.2 particulate=break up of solid combust fossil fuel voltaic active forest fire biological voc=petroleum ch4 bacteria decomposition,1
4023,5714,forest%20fire,NaN,how is it one careless match can start a forest fire but it takes a whole box to start a campfire?,1
4024,5715,forest%20fire,port matilda pa,our little forest fire wardens http://t.co/aprensss3x,1
4031,5724,forest%20fire,NaN,it starts a forest fire that cannot be put out. http://t.co/3stfmn26r9,1
4033,5727,forest%20fire,NaN,i had to grill for a school function. one of the grills we had going was pretty much either off or forest fire. no inbetween! made it work,0
4034,5729,forest%20fire,NaN,5:15p for those who watch over me forest fire smoke has lifted 'nuff f/me t/make a run t/the pet store on the hwy (n). i'll b/home by 7p.,1
4035,5730,forest%20fire,Los Angeles for now,we are having forest fires out here in #socal but the only fire you should watch for is @ybtheprophet #realhiphop https://t.co/zpsnx5iyaj,1
4038,5734,forest%20fire,Atlanta,how is it one careless match can start a forest fire but it takes a whole box to start a campfire,0
4039,5736,forest%20fires,NaN,only you can prevent forest fires. ???? http://t.co/rgyuakc0dr,1
4041,5740,forest%20fires,NaN,q: why do ducks have big flat feet? a: to stamp out forest fires. q: why do elephants have big flat feet? a: to stamp out flaming ducks.,0


In [69]:
train[(train['text'].str.contains('#'))]

,id,keyword,location,text,target
0,1,NaN,NaN,our deeds are the reason of this #earthquake may allah forgive us all,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation orders in california",1
4,7,NaN,NaN,just got sent this photo from ruby #alaska as smoke from #wildfires pours into a school,1
5,8,NaN,NaN,#rockyfire update => california hwy. 20 closed in both directions due to lake county fire - #cafire #wildfires,1
6,10,NaN,NaN,"#flood #disaster heavy rain causes flash flooding of streets in manitou, colorado springs areas",1
...,...,...,...,...,...
7598,10853,NaN,NaN,father-of-three lost control of car after overtaking and collided #bathandnortheastsomerset http://t.co/fa3fcnln86,1
7599,10854,NaN,NaN,1.3 #earthquake in 9km ssw of anza california #iphone users download the earthquake app for more information http://t.co/v3azwoamzk,1
7601,10859,NaN,NaN,#breaking #la refugio oil spill may have been costlier bigger than projected http://t.co/5uecmcv2pk,1
7604,10863,NaN,NaN,#worldnews fallen powerlines on g:link tram: update: fire crews have evacuated up to 30 passengers who were tr... http://t.co/eysvvza7qm,1


In [37]:
train['text'] = train['text'].str.lower()
train['keyword'] = train['keyword'].str.lower()

In [59]:
len('6ioabsl6i7')

10

In [60]:
test = pd.read_csv('./data/test.csv')
test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, stay safe everyone."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all"
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [64]:
train[train['text'].duplicated()]

,id,keyword,location,text,target
48,68,ablaze,Live On Webcam,check these out: http://t.co/roi2nsmejj http://t.co/3tj8zjin21 http://t.co/yduixefipe http://t.co/lxtjc87kls #nsfw,0
115,165,aftershock,US,320 [ir] icemoon [aftershock] | http://t.co/vam5podgyw | @djicemoon | #dubstep #trapmusic #dnb #edm #dance #icesû_ http://t.co/zevakjapcz,0
119,172,aftershock,Switzerland,320 [ir] icemoon [aftershock] | http://t.co/thyzomvwu0 | @djicemoon | #dubstep #trapmusic #dnb #edm #dance #icesû_ http://t.co/83joo0xk29,0
164,238,airplane%20accident,NaN,experts in france begin examining airplane debris found on reunion island: french air accident experts on wednesday began examining t...,1
624,898,bioterrorism,NaN,to fight bioterrorism sir.,0
...,...,...,...,...,...
7600,10855,NaN,NaN,evacuation order lifted for town of roosevelt: http://t.co/edyfo6e2pu http://t.co/m5kxlpkfa1,1
7607,10867,NaN,NaN,#stormchase violent record breaking ef-5 el reno oklahoma tornado nearly runs over ... - http://t.co/3sicroaanz http://t.co/i27oa0hisp,1
7609,10870,NaN,NaN,@aria_ahrary @thetawniest the out of control wild fires in california even in the northern part of the state. very troubling.,1
7610,10871,NaN,NaN,m1.94 [01:04 utc]?5km s of volcano hawaii. http://t.co/zdtoyd8ebj,1


In [66]:
test['text'].duplicated().sum()

20

In [75]:
train[(train['text'].str.contains('t.co'))]['target'].value_counts(normalize=True)

1    0.543634
0    0.456366
Name: target, dtype: float64

In [73]:
train['target'].value_counts(normalize=True)

0    0.57034
1    0.42966
Name: target, dtype: float64

In [147]:
train[(train['text'].str.contains('http'))&~(train['text'].str.contains('t.co'))]

,id,keyword,location,text,target,text_nourl
2395,3446,derail,NaN,dozens die as two trains derail into a river in indiahttp://www.informationng.com/?p=309943,1,dozens die as two trains derail into a river in


In [116]:
train[train['keyword'].str.contains(' ')]

,id,keyword,location,text,target


In [88]:
keyword_dis = {}
for keyword in set(train['keyword']):
    keyword_dis[keyword] = train[train['keyword'] == keyword]['target'].mean()

In [95]:
for key, value in keyword_dis.items():
    if value > .9:
        print(f'{key} : {value:.4}')

suicide%20bomb : 0.9143
bombing : 0.931
rescuers : 0.9143
derailment : 1.0
suicide%20bomber : 0.9677
wreckage : 1.0
oil%20spill : 0.9737
suicide%20bombing : 0.9697
nuclear%20disaster : 0.9118
typhoon : 0.9737
outbreak : 0.975
debris : 1.0


In [94]:
for key, value in keyword_dis.items():
    if value < .25:
        print(f'{key} : {value:.4}')

armageddon : 0.119
panicking : 0.06061
traumatised : 0.05714
arsonist : 0.2353
blaze : 0.1316
burned : 0.2121
trouble : 0.1515
riot : 0.1765
battle : 0.1923
meltdown : 0.1515
army : 0.1471
collide : 0.08824
obliteration : 0.1379
crush : 0.1081
explode : 0.07895
demolish : 0.1471
inundated : 0.1714
smoke : 0.08824
inundation : 0.2
deluge : 0.1429
flattened : 0.1471
destruction : 0.1765
body%20bag : 0.0303
blown%20up : 0.1515
harm : 0.09756
desolation : 0.1944
mayhem : 0.1
catastrophe : 0.1944
curfew : 0.1351
body%20bags : 0.02439
upheaval : 0.1579
panic : 0.08108
blazing : 0.02941
destroy : 0.2432
screaming : 0.05556
threat : 0.1818
twister : 0.125
hazard : 0.2059
survive : 0.1562
military : 0.2353
blight : 0.0625
fear : 0.125
dead : 0.2333
ruin : 0.02703
wrecked : 0.07692
wreck : 0.1892
stretcher : 0.09091
danger : 0.2222
screams : 0.1429
pandemonium : 0.2162
sirens : 0.2414
drown : 0.09375
lava : 0.1471
hellfire : 0.1795
quarantine : 0.1471
cliff%20fall : 0.2222
aftershock : 0.0
sunk 

In [101]:
train['keyword'].fillna('-', inplace=True)

In [112]:
i = train['text'].str.find('t.co')

train.iloc[68]['text'].slice[6,9]

AttributeError: 'str' object has no attribute 'slice'

In [139]:
for i in train['text'][:100]:
    if 'https' in i:
        locat = i.find('https')
        print(i[locat:locat+23])       
    elif 'http' in i:
        locat = i.find('http')
        print(i[locat:locat+22])      

http://t.co/lhyxeohy6c
http://t.co/yao1e0xngw
http://t.co/2nndbgwyei
http://t.co/qqsmshaj3n
http://t.co/3imaomknna
http://t.co/wdueaj8q4j
http://t.co/roi2nsmejj
http://t.co/o9qknbfofx
http://t.co/tiyulif5c6
http://t.co/vl5tbr3wbr
http://t.co/roi2nsmejj
http://t.co/gwnrhmso8k
http://t.co/ev1ahoucza
http://t.co/vplr5hka2u
http://t.co/pcxarbh9an
http://t.co/uc4j4jhvgr
http://t.co/4izfdyc3cg
http://t.co/8kscqkfkkf
https://t.co/cyompz1a0z
http://t.co/icsjgz9te1
http://t.co/9fxpixqujt
https://t.co/7mlmsuzv1z
http://t.co/av2jsjfftc
http://t.co/rwzbfvnxer
https://t.co/0ghk693egj
http://t.co/pmlohzurwr
http://t.co/gkye6gjtk5
https://t.co/k4zomof319
http://t.co/ybjezkurw1
http://t.co/7s9nm1fict
https://t.co/ysxun5vceh
http://t.co/rs22lj4qfp
http://t.co/hylmo0wgfi
http://t.co/p18lzrlmy6
http://t.co/oia5fxi4gm
http://t.co/vckit6edev
http://t.co/6otfp31lqw
https://t.co/bptmlf4p10


In [142]:
URLless_string = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', i)
print(URLless_string)

only had a car for not even a week and got in a fucking car accident .. mfs can't fucking drive .


In [143]:
# Using this string remover from StackOverflow: 
# https://stackoverflow.com/questions/11331982/how-to-remove-any-url-within-a-string-in-python
train['text_nourl'] = [re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', i) for i in train['text']]

In [146]:
train[train['text'].str.contains('http')].head()

,id,keyword,location,text,target,text_nourl
31,48,ablaze,Birmingham,@bbcmtd wholesale markets ablaze http://t.co/lhyxeohy6c,1,@bbcmtd wholesale markets ablaze
32,49,ablaze,Est. September 2012 - Bristol,we always try to bring the heavy. #metal #rt http://t.co/yao1e0xngw,0,we always try to bring the heavy. #metal #rt
33,50,ablaze,AFRICA,#africanbaze: breaking news:nigeria flag set ablaze in aba. http://t.co/2nndbgwyei,1,#africanbaze: breaking news:nigeria flag set ablaze in aba.
35,53,ablaze,"London, UK",on plus side look at the sky last night it was ablaze http://t.co/qqsmshaj3n,0,on plus side look at the sky last night it was ablaze
37,55,ablaze,World Wide!!,inec office in abia set ablaze - http://t.co/3imaomknna,1,inec office in abia set ablaze -


In [149]:
train['text_nourl'].duplicated().sum()

634

,id,keyword,location,text,target,text_nourl
0,1,-,NaN,our deeds are the reason of this #earthquake may allah forgive us all,1,our deeds are the reason of this #earthquake may allah forgive us all
1,4,-,NaN,forest fire near la ronge sask. canada,1,forest fire near la ronge sask. canada
2,5,-,NaN,all residents asked to 'shelter in place' are being notified by officers. no other evacuation or shelter in place orders are expected,1,all residents asked to 'shelter in place' are being notified by officers. no other evacuation or shelter in place orders are expected
3,6,-,NaN,"13,000 people receive #wildfires evacuation orders in california",1,"13,000 people receive #wildfires evacuation orders in california"
4,7,-,NaN,just got sent this photo from ruby #alaska as smoke from #wildfires pours into a school,1,just got sent this photo from ruby #alaska as smoke from #wildfires pours into a school
...,...,...,...,...,...,...
7602,10860,-,NaN,a siren just went off and it wasn't the forney tornado warning ??,1,a siren just went off and it wasn't the forney tornado warning ??
7603,10862,-,NaN,officials say a quarantine is in place at an alabama home over a possible ebola case after developing symptoms... http://t.co/rqkk15uhey,1,officials say a quarantine is in place at an alabama home over a possible ebola case after developing symptoms...
7604,10863,-,NaN,#worldnews fallen powerlines on g:link tram: update: fire crews have evacuated up to 30 passengers who were tr... http://t.co/eysvvza7qm,1,#worldnews fallen powerlines on g:link tram: update: fire crews have evacuated up to 30 passengers who were tr...
7605,10864,-,NaN,on the flip side i'm at walmart and there is a bomb and everyone had to evacuate so stay tuned if i blow up or not,1,on the flip side i'm at walmart and there is a bomb and everyone had to evacuate so stay tuned if i blow up or not


In [155]:
train['keyword'] = train['keyword'].str.replace('%20', ' ')

In [156]:
keywords_sorted = sorted(set(train['keyword']))
keywords_sorted

['-',
 'ablaze',
 'accident',
 'aftershock',
 'airplane accident',
 'ambulance',
 'annihilated',
 'annihilation',
 'apocalypse',
 'armageddon',
 'army',
 'arson',
 'arsonist',
 'attack',
 'attacked',
 'avalanche',
 'battle',
 'bioterror',
 'bioterrorism',
 'blaze',
 'blazing',
 'bleeding',
 'blew up',
 'blight',
 'blizzard',
 'blood',
 'bloody',
 'blown up',
 'body bag',
 'body bagging',
 'body bags',
 'bomb',
 'bombed',
 'bombing',
 'bridge collapse',
 'buildings burning',
 'buildings on fire',
 'burned',
 'burning',
 'burning buildings',
 'bush fires',
 'casualties',
 'casualty',
 'catastrophe',
 'catastrophic',
 'chemical emergency',
 'cliff fall',
 'collapse',
 'collapsed',
 'collide',
 'collided',
 'collision',
 'crash',
 'crashed',
 'crush',
 'crushed',
 'curfew',
 'cyclone',
 'damage',
 'danger',
 'dead',
 'death',
 'deaths',
 'debris',
 'deluge',
 'deluged',
 'demolish',
 'demolished',
 'demolition',
 'derail',
 'derailed',
 'derailment',
 'desolate',
 'desolation',
 'destroy',

In [158]:
lemmatizer = WordNetLemmatizer()
keywords_lem = []
for word in keywords_sorted:
    keywords_lem.append(lemmatizer.lemmatize(word))
keywords_lem

['-',
 'ablaze',
 'accident',
 'aftershock',
 'airplane accident',
 'ambulance',
 'annihilated',
 'annihilation',
 'apocalypse',
 'armageddon',
 'army',
 'arson',
 'arsonist',
 'attack',
 'attacked',
 'avalanche',
 'battle',
 'bioterror',
 'bioterrorism',
 'blaze',
 'blazing',
 'bleeding',
 'blew up',
 'blight',
 'blizzard',
 'blood',
 'bloody',
 'blown up',
 'body bag',
 'body bagging',
 'body bags',
 'bomb',
 'bombed',
 'bombing',
 'bridge collapse',
 'buildings burning',
 'buildings on fire',
 'burned',
 'burning',
 'burning buildings',
 'bush fires',
 'casualty',
 'casualty',
 'catastrophe',
 'catastrophic',
 'chemical emergency',
 'cliff fall',
 'collapse',
 'collapsed',
 'collide',
 'collided',
 'collision',
 'crash',
 'crashed',
 'crush',
 'crushed',
 'curfew',
 'cyclone',
 'damage',
 'danger',
 'dead',
 'death',
 'death',
 'debris',
 'deluge',
 'deluged',
 'demolish',
 'demolished',
 'demolition',
 'derail',
 'derailed',
 'derailment',
 'desolate',
 'desolation',
 'destroy',
 '

In [162]:
lemmatizer.lemmatize('bloods')

'blood'

In [165]:
comp = list(zip(keywords_lem, keywords_sorted))
for i in comp:
    if i[0] == i[1]:
        print(i)

('-', '-')
('ablaze', 'ablaze')
('accident', 'accident')
('aftershock', 'aftershock')
('airplane accident', 'airplane accident')
('ambulance', 'ambulance')
('annihilated', 'annihilated')
('annihilation', 'annihilation')
('apocalypse', 'apocalypse')
('armageddon', 'armageddon')
('army', 'army')
('arson', 'arson')
('arsonist', 'arsonist')
('attack', 'attack')
('attacked', 'attacked')
('avalanche', 'avalanche')
('battle', 'battle')
('bioterror', 'bioterror')
('bioterrorism', 'bioterrorism')
('blaze', 'blaze')
('blazing', 'blazing')
('bleeding', 'bleeding')
('blew up', 'blew up')
('blight', 'blight')
('blizzard', 'blizzard')
('blood', 'blood')
('bloody', 'bloody')
('blown up', 'blown up')
('body bag', 'body bag')
('body bagging', 'body bagging')
('body bags', 'body bags')
('bomb', 'bomb')
('bombed', 'bombed')
('bombing', 'bombing')
('bridge collapse', 'bridge collapse')
('buildings burning', 'buildings burning')
('buildings on fire', 'buildings on fire')
('burned', 'burned')
('burning', 'b

In [170]:
pstem = PorterStemmer()
keywords_stem = [pstem.stem(i) for i in keywords_sorted]
comp2 = list(zip(keywords_stem, keywords_sorted))
for i in comp2:
    if i[0] != i[1]:
        print(i)

('ablaz', 'ablaze')
('accid', 'accident')
('airplane accid', 'airplane accident')
('ambul', 'ambulance')
('annihil', 'annihilated')
('annihil', 'annihilation')
('apocalyps', 'apocalypse')
('armi', 'army')
('attack', 'attacked')
('avalanch', 'avalanche')
('battl', 'battle')
('bioterror', 'bioterrorism')
('blaze', 'blazing')
('bleed', 'bleeding')
('bloodi', 'bloody')
('body bag', 'body bagging')
('body bag', 'body bags')
('bomb', 'bombed')
('bomb', 'bombing')
('bridge collaps', 'bridge collapse')
('buildings burn', 'buildings burning')
('buildings on fir', 'buildings on fire')
('burn', 'burned')
('burn', 'burning')
('burning build', 'burning buildings')
('bush fir', 'bush fires')
('casualti', 'casualties')
('casualti', 'casualty')
('catastroph', 'catastrophe')
('catastroph', 'catastrophic')
('chemical emerg', 'chemical emergency')
('cliff fal', 'cliff fall')
('collaps', 'collapse')
('collaps', 'collapsed')
('collid', 'collide')
('collid', 'collided')
('collis', 'collision')
('crash', 'cr

In [176]:
snowstem = SnowballStemmer('english')

train['keywords_stemmed'] = train['keyword'].map(lambda x: snowstem.stem(x))
train['keywords_stemmed'].value_counts()

In [180]:
keyword_dis1 = {}
for keyword in set(train['keywords_stemmed']):
    keyword_dis[keyword] = train[train['keywords_stemmed'] == keyword]['target'].mean()
for key, value in keyword_dis.items():
    if value > .8:
        print(f'{key} : {value:.4}')

suicide%20bomb : 0.9143
bombing : 0.931
rescuers : 0.9143
wildfire : 0.8788
derailment : 1.0
airplane%20accident : 0.8571
suicide%20bomber : 0.9677
mass%20murder : 0.8485
evacuated : 0.8889
razed : 0.8857
bridge%20collapse : 0.8286
wreckage : 1.0
oil%20spill : 0.9737
wild%20fires : 0.871
suicide%20bombing : 0.9697
forest%20fires : 0.8438
nuclear%20disaster : 0.9118
typhoon : 0.9737
outbreak : 0.975
debris : 1.0
buildings%20on%20fire : 0.8485
bridge collaps : 0.8286
nuclear disast : 0.9118
raze : 0.8857
wild fir : 0.871
forest fir : 0.8235
wreckag : 1.0
mass murd : 0.8485
suicide bomb : 0.9495
wildfir : 0.8788
buildings on fir : 0.8485
oil spil : 0.9737
debri : 1.0
rescuer : 0.9143
airplane accid : 0.8571


In [203]:

no_key = train.loc[train['keyword'] == '-']
for index, row in no_key.iterrows():
    for word in keywords_sorted:
        if word in row['text']:
            train.loc[index, 'keyword'] = word
train[train['keyword'] == '-']

,id,keyword,location,text,target,text_nourl,keywords_stemmed
15,23,-,NaN,what's up man?,0,what's up man?,-
16,24,-,NaN,i love fruits,0,i love fruits,-
17,25,-,NaN,summer is lovely,0,summer is lovely,-
18,26,-,NaN,my car is so fast,0,my car is so fast,-
19,28,-,NaN,what a goooooooaaaaaal!!!!!!,0,what a goooooooaaaaaal!!!!!!,-
20,31,-,NaN,this is ridiculous....,0,this is ridiculous....,-
21,32,-,NaN,london is cool ;),0,london is cool ;),-
22,33,-,NaN,love skiing,0,love skiing,-
23,34,-,NaN,what a wonderful day!,0,what a wonderful day!,-
24,36,-,NaN,looooool,0,looooool,-
